In [1]:
import os
from glob import glob

#Change directory
os.chdir("D:\Course Learning - UM\Sem 5\Data Science Project\Emotion Analysis for Long Covid\Datasets") 

#Basic library
import pandas as pd
import numpy as np

#Data Cleaning Library/RegEx
import re
import neattext as nt
import neattext.functions as nfx  #https://github.com/Jcharis/neattext
#!pip install neattext

# Detect language library
from textblob import TextBlob #Option 1 - Error HTTP 429 Too many request ()
from langdetect import detect, DetectorFactory #Option 2 - OK
#!pip install textblob
#!pip install langdetect

#Data visualization library
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

#Others
from datetime import datetime 
import time

Check directory use
`pwd`
`ls`

In [1]:
pwd

'C:\\Users\\User\\My Jupyter Notebook'

In [ ]:
#Load Text Cleaning Package
#Methods
dir(nfx)

##### Use `glob()` to list all files that match a pattern and sort results

In [29]:
filenames = sorted(glob('LongCovid_*.csv'))
filenames

['LongCovid_5May.csv',
 'LongCovid_6June.csv',
 'LongCovid_7July.csv',
 'LongCovid_8August.csv',
 'LongCovid_9September.csv']

In [30]:
#Read the datasets
df = pd.concat((pd.read_csv(f) for f in filenames),ignore_index=True)

#pd.concat((pd.read_csv(file).assign(filename = file) for file in files), ignore_index=True)


In [31]:
df

,Datetime,Tweet Id,Text,Username,Number of Likes
0,2021-05-30 23:59:49+00:00,1399153605588836354,As Australia begins the long journey to post-C...,LexiconDigital_,1
1,2021-05-30 23:59:29+00:00,1399153522545942531,"@amyklobuchar @All100Senators Its not over, ev...",bargdaffy161,0
2,2021-05-30 23:59:00+00:00,1399153402643288064,@JoeBarri Because not all people have had the ...,Miss_Naste,0
3,2021-05-30 23:58:51+00:00,1399153364311724033,COVID shone a spotlight on for-profit long ter...,JSSBennett,0
4,2021-05-30 23:58:37+00:00,1399153305218170882,@kiroriwalanuj @swamiyogeshji @c_aashish @bhar...,Vl17262873,2
...,...,...,...,...,...
130000,2021-09-24 18:36:49+00:00,1441471699510300674,@CiNNeR_ @WheelieNick @ZelfBart Vertel mij een...,Net_Arts,0
130001,2021-09-24 18:36:30+00:00,1441471622796365824,Coronavirus latest: US’s top medical research ...,COVID19LiveNow,0
130002,2021-09-24 18:36:25+00:00,1441471599560077316,@csmalls11 @WFKARS Sorry! I am taking back eve...,CowChickEnCow,2
130003,2021-09-24 18:36:21+00:00,1441471585567666176,"2/noted the statement, which doesn’t commit to...",ljfawkes,0


## --------------------------------------Output The combine Datasets-------------------------------------------

In [32]:
df.to_csv('D:\\Course Learning - UM\\Sem 5\\Data Science Project\\Emotion Analysis for Long Covid\\testing clean text\\Combine5Months.csv',
          columns = ['Datetime','Tweet Id', 'Text', 'Username', 'Number of Likes'], index= False)

# -----------------------------------------------------------------------------------------------------------

In [63]:
# Show duplicated
df.duplicated('Text', keep = 'first').value_counts()

False    97102
dtype: int64

In [64]:
duplicated = df[df.duplicated('Text')]
duplicated.head(10)

,Datetime,Tweet Id,Text,Username,Number of Likes,clean_tweet,emotion


In [13]:
df['Text'].iloc[29564]

'#コロナ後遺症 (疑い) (#Longcovid) \n\n承認なし、入退室自由、雑談もスタンプも歓迎❕\nリアルで話せない後遺症のしんどさ、改善した方法、日々の過ごし方を共有しながら、機嫌よく過ごせるトークを皆で考えよう\n\nオープンチャット「ｺﾛﾅ後遺症(疑)だけど機嫌よく生きるよ」\nhttps://t.co/evh7gFYZqp'

In [14]:
df['Text'].iloc[67]

'@LongCovidKids @LongCovidSOS @longcovidaragon @covideuskal @long_covid @LongCovidItalia @LongCovidWales @LongCovidNL @covid_madrid'

In [15]:
#Data Cleaning

def cleanTxt(text):
    
    if re.match(r'RT[\s]+|@RT[\s]+', text): #Removing RT @RT
        return True
    else:
        return False

def countWords(text):
    
    counts = len(text.split())
    return counts

def removeFewerWords(text):
    
    counts = len(text.split())
    if counts < 4:
        return True

def detlang(text):
    
    return detect(text)
    #return(TextBlob(text).detect_language())        

#Calculate the time used
start = datetime.now()

#Show size before cleaning
print("Size before cleaning: ", df.shape)

#Customize cleaning
df.drop(index=df[df['Text'].apply(cleanTxt)==True].index, inplace = True) #Remove RT

#Cleaning text
df['clean_tweet'] = df['Text'].apply(nfx.remove_hashtags) #remove whole #Hashtags
df['clean_tweet'] = df['clean_tweet'].apply(lambda x: nfx.remove_userhandles(x)) #remove @mention
df['clean_tweet'] = df['clean_tweet'].apply(nfx.remove_multiple_spaces) 
df['clean_tweet'] = df['clean_tweet'].apply(nfx.remove_urls)
df['clean_tweet'] = df['clean_tweet'].apply(nfx.remove_emails)
df['clean_tweet'] = df['clean_tweet'].apply(nfx.remove_numbers)
df['clean_tweet'] = df['clean_tweet'].apply(nfx.remove_emojis)

#---Data Pre-processing---
df['clean_tweet'] = df['clean_tweet'].apply(nfx.remove_puncts) #remove punctuations
df['clean_tweet'] = df['clean_tweet'].apply(nfx.remove_special_characters) #remove special characters
df['clean_tweet'] = df['clean_tweet'].apply(nfx.remove_non_ascii)
df['clean_tweet'] = df['clean_tweet'].str.lower() #Convert upper case to lower case
df['clean_tweet'] = df['clean_tweet'].apply(nfx.remove_stopwords)


#remove dupluicates
df.drop_duplicates(subset = "clean_tweet", keep = 'first', inplace = True)

#Remove shorts tweets
df.drop(index=df[df['clean_tweet'].apply(removeFewerWords)==True].index, inplace = True)
    
# Detect & Remove languague others than English
DetectorFactory.seed = 0 #Enforce consistent results
df['language'] = df['clean_tweet'].apply(detlang)
df.drop(df[df.language != 'en'].index, inplace=True)

#Show size after cleaning
print("Size after cleaning: ", df.shape)

stop = datetime.now()
execution_time = stop-start 
print("Execution time is: ", execution_time)


Size before cleaning:  (130005, 5)
Size after cleaning:  (97102, 7)
Execution time is:  0:21:11.354893


In [17]:
df.head()

,Datetime,Tweet Id,Text,Username,Number of Likes,clean_tweet,language
0,2021-05-30 23:59:49+00:00,1399153605588836354,As Australia begins the long journey to post-C...,LexiconDigital_,1,australia begins long journey postcovid recove...,en
1,2021-05-30 23:59:29+00:00,1399153522545942531,"@amyklobuchar @All100Senators Its not over, ev...",bargdaffy161,0,heard long covid hot mess senator,en
2,2021-05-30 23:59:00+00:00,1399153402643288064,@JoeBarri Because not all people have had the ...,Miss_Naste,0,people opportunity vaccine area high percentag...,en
3,2021-05-30 23:58:51+00:00,1399153364311724033,COVID shone a spotlight on for-profit long ter...,JSSBennett,0,covid shone spotlight forprofit long term care...,en
4,2021-05-30 23:58:37+00:00,1399153305218170882,@kiroriwalanuj @swamiyogeshji @c_aashish @bhar...,Vl17262873,2,yes agree anil sensible looks longterm neglect...,en


In [18]:
sample = 'Text'
duplicated = df[df.duplicated(sample)]
print(df.duplicated(sample, keep = 'first').value_counts())
duplicated.head(10)

False    97102
dtype: int64


,Datetime,Tweet Id,Text,Username,Number of Likes,clean_tweet,language


In [19]:
# Output Clean Data
df.to_csv('D:\\Course Learning - UM\\Sem 5\\Data Science Project\\Emotion Analysis for Long Covid\\testing clean text\\CleanDataV4.csv', 
          columns = ['Datetime','Tweet Id','Text','Username','Number of Likes','clean_tweet','language'], index= False)

# -------------------------Read CleanData For Emotion Labelling------------------------

In [ ]:
pwd

In [2]:
df = pd.read_csv("D:\\Course Learning - UM\\Sem 5\\Data Science Project\\Emotion Analysis for Long Covid\\testing clean text\\CleanDataV4.csv")

In [58]:
df

,Datetime,Tweet Id,Text,Username,Number of Likes,clean_tweet,language
0,2021-05-30 23:59:49+00:00,1399153605588836354,As Australia begins the long journey to post-C...,LexiconDigital_,1,australia begins long journey postcovid recove...,en
1,2021-05-30 23:59:29+00:00,1399153522545942531,"@amyklobuchar @All100Senators Its not over, ev...",bargdaffy161,0,heard long covid hot mess senator,en
2,2021-05-30 23:59:00+00:00,1399153402643288064,@JoeBarri Because not all people have had the ...,Miss_Naste,0,people opportunity vaccine area high percentag...,en
3,2021-05-30 23:58:51+00:00,1399153364311724033,COVID shone a spotlight on for-profit long ter...,JSSBennett,0,covid shone spotlight forprofit long term care...,en
4,2021-05-30 23:58:37+00:00,1399153305218170882,@kiroriwalanuj @swamiyogeshji @c_aashish @bhar...,Vl17262873,2,yes agree anil sensible looks longterm neglect...,en
...,...,...,...,...,...,...,...
97097,2021-09-24 18:37:18+00:00,1441471824500432897,Spent the day in BITS Pilani. Just reached Del...,shantanunandan2,85,spent day bits pilani reached delhi semesterlo...,en
97098,2021-09-24 18:36:30+00:00,1441471622796365824,Coronavirus latest: US’s top medical research ...,COVID19LiveNow,0,coronavirus latest uss medical research agency...,en
97099,2021-09-24 18:36:25+00:00,1441471599560077316,@csmalls11 @WFKARS Sorry! I am taking back eve...,CowChickEnCow,2,sorry taking tweet massive karenoutburst schoo...,en
97100,2021-09-24 18:36:21+00:00,1441471585567666176,"2/noted the statement, which doesn’t commit to...",ljfawkes,0,noted statement doesnt commit providing noninf...,en


In [96]:
df1 = df[27500:30000]

In [97]:
df1

,Datetime,Tweet Id,Text,Username,Number of Likes,clean_tweet,language
27500,2021-06-28 22:07:26+00:00,1409634574494146560,"@Therese_Rein Like all those with ME and CFS, ...",SunflowersChai,0,like cfs long covid sufferers left live non li...,en
27501,2021-06-28 22:06:39+00:00,1409634376564961284,@Pladizow well thankfully like all vaccines th...,forkinggizmo,0,thankfully like vaccines selfish protecting se...,en
27502,2021-06-28 22:06:00+00:00,1409634210894123020,Nick Guthe says he doesn’t judge his wife for ...,hereandnow,7,nick guthe says doesnt judge wife taking life ...,en
27503,2021-06-28 22:05:33+00:00,1409634100508459010,Read this and weep. Then write to your MP and ...,ClaireHastie1,5,read weep write mp use voice social media push...,en
27504,2021-06-28 22:05:33+00:00,1409634099187257344,"@normaburke @adriancummins To be honest, I had...",GerryMckeever2,0,honest leave long term employment donegal enti...,en
...,...,...,...,...,...,...,...
29995,2021-06-28 14:36:53+00:00,1409521188620079108,@NickTriggle Can change due to vaccine escape ...,Twirrety,2,change vaccine escape case numbers data dont l...,en
29996,2021-06-28 14:36:45+00:00,1409521155153747968,The @NCAA is trash for this. As long as you ge...,austincoIweII,1,trash long revenue rolling covid doesnt exist ehh,en
29997,2021-06-28 14:36:35+00:00,1409521113323827205,"Latest #Health #News: ""Chronic fatigue syndrom...",CaramelParsley,0,latest chronic fatigue syndrome moving controv...,en
29998,2021-06-28 14:36:05+00:00,1409520984999215105,Two kids test positive for COVID-19 during Roy...,CynthiaFlynn15,0,kids test positive covid royal caribbean cruis...,en


# ----------------------------------------IBM Watson-----------------------------------------------

In [ ]:
!pip install --upgrade "ibm-watson>=5.3.0"

In [ ]:
pip list

# Authenticate

In [98]:
import json
from ibm_watson import ToneAnalyzerV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

version = '2021-11-17'
ibmApiKey = 'qJajmNiiES2SAecQxIB-OtqpkFF6w7NG5O-bXAIsQc9V'
ibmUrl = 'https://api.au-syd.tone-analyzer.watson.cloud.ibm.com/instances/6ef68df0-2d53-465d-8938-319738756b39'

authenticator = IAMAuthenticator(ibmApiKey)
tone_analyzer = ToneAnalyzerV3(
    version= version,
    authenticator=authenticator
)
tone_analyzer.set_service_url(ibmUrl)


# Analyze Tone

In [100]:
def detectEmotion(text):

    tone_analysis = tone_analyzer.tone(text).get_result()
    if len(tone_analysis['document_tone']['tones'])==0:
        return 'noTone'
    
    elif len(tone_analysis['document_tone']['tones'])==1:
        return str(tone_analysis['document_tone']['tones'][0]['tone_id'])
    
    #Find the maximum value of emotion
    elif len(tone_analysis['document_tone']['tones'])>1: 
        num1 = len(tone_analysis['document_tone']['tones'])
        maxIndex = 0
        maxScore = tone_analysis['document_tone']['tones'][0]['score']
        for num in range(num1):
            temp = tone_analysis['document_tone']['tones'][num]['score']
            if temp > maxScore:
                maxIndex = num
                maxScore = temp
        return str(tone_analysis['document_tone']['tones'][maxIndex]['tone_id'])

start = datetime.now()
print("Starting......")

df1['emotion'] = df1['clean_tweet'].apply(detectEmotion)

stop = datetime.now()
execution_time = stop-start 
print("Execution time is: ", execution_time)

Starting......
Execution time is:  0:24:28.956766


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [101]:
df1['clean_tweet'].iloc[9]

'photos mistress problem scorned wife battling long covid got hancock going'

In [102]:
df1

,Datetime,Tweet Id,Text,Username,Number of Likes,clean_tweet,language,emotion
27500,2021-06-28 22:07:26+00:00,1409634574494146560,"@Therese_Rein Like all those with ME and CFS, ...",SunflowersChai,0,like cfs long covid sufferers left live non li...,en,sadness
27501,2021-06-28 22:06:39+00:00,1409634376564961284,@Pladizow well thankfully like all vaccines th...,forkinggizmo,0,thankfully like vaccines selfish protecting se...,en,confident
27502,2021-06-28 22:06:00+00:00,1409634210894123020,Nick Guthe says he doesn’t judge his wife for ...,hereandnow,7,nick guthe says doesnt judge wife taking life ...,en,joy
27503,2021-06-28 22:05:33+00:00,1409634100508459010,Read this and weep. Then write to your MP and ...,ClaireHastie1,5,read weep write mp use voice social media push...,en,sadness
27504,2021-06-28 22:05:33+00:00,1409634099187257344,"@normaburke @adriancummins To be honest, I had...",GerryMckeever2,0,honest leave long term employment donegal enti...,en,noTone
...,...,...,...,...,...,...,...,...
29995,2021-06-28 14:36:53+00:00,1409521188620079108,@NickTriggle Can change due to vaccine escape ...,Twirrety,2,change vaccine escape case numbers data dont l...,en,noTone
29996,2021-06-28 14:36:45+00:00,1409521155153747968,The @NCAA is trash for this. As long as you ge...,austincoIweII,1,trash long revenue rolling covid doesnt exist ehh,en,noTone
29997,2021-06-28 14:36:35+00:00,1409521113323827205,"Latest #Health #News: ""Chronic fatigue syndrom...",CaramelParsley,0,latest chronic fatigue syndrome moving controv...,en,fear
29998,2021-06-28 14:36:05+00:00,1409520984999215105,Two kids test positive for COVID-19 during Roy...,CynthiaFlynn15,0,kids test positive covid royal caribbean cruis...,en,confident


In [103]:
df1['emotion'].value_counts()

analytical    902
sadness       498
noTone        461
tentative     248
joy           137
confident     124
fear           89
anger          41
Name: emotion, dtype: int64

In [107]:
#Output the labelled datasets
df1.to_csv('D:\\Course Learning - UM\\Sem 5\\Data Science Project\\Emotion Analysis for Long Covid\\LabelEmotion\\Max\\Emotion12-27500-29999.csv', 
          columns = ['Datetime','Tweet Id','Text','Username','Number of Likes','clean_tweet','language','emotion'], index= False)

# ————————————Combine Label Datasets————————————

In [40]:
os.chdir("D:\Course Learning - UM\Sem 5\Data Science Project\Emotion Analysis for Long Covid\LabelEmotion\Max")
filenames = sorted(glob('Emotion*.csv'))
filenames

['Emotion01-0-2499.csv',
 'Emotion02-2500-4999.csv',
 'Emotion03-5000-7499.csv',
 'Emotion04-7500-9999.csv',
 'Emotion05-10000-12499.csv',
 'Emotion06-12500-14999.csv',
 'Emotion07-15000-17499.csv',
 'Emotion08-17500-19999.csv',
 'Emotion09-20000-22499.csv',
 'Emotion10-22500-24999.csv',
 'Emotion11-25000-27499.csv',
 'Emotion12-27500-29999.csv',
 'Emotion13-30000-32499.csv',
 'Emotion14-32500-34999.csv',
 'Emotion15-35000-37499.csv',
 'Emotion16-37500-39999.csv',
 'Emotion17-40000-42499.csv',
 'Emotion18-42500-44999.csv',
 'Emotion19-45000-47499.csv',
 'Emotion20-47500-49999.csv',
 'Emotion21-50000-52499.csv',
 'Emotion22-52500-54999.csv',
 'Emotion23-55000-57499.csv',
 'Emotion24-57500-59999.csv',
 'Emotion25-60000-62499.csv',
 'Emotion26-62500-64999.csv',
 'Emotion27-65000-67499.csv',
 'Emotion28-67500-69999.csv',
 'Emotion29-70000-72499.csv',
 'Emotion30-72500-74999.csv',
 'Emotion31-75000-77499.csv',
 'Emotion32-77500-79999.csv',
 'Emotion33-80000-82499.csv',
 'Emotion34-82500-849

In [41]:
#Read the datasets
df = pd.concat((pd.read_csv(f) for f in filenames),ignore_index=True)

In [42]:
df

,Datetime,Tweet Id,Text,Username,Number of Likes,clean_tweet,language,emotion
0,2021-05-30 23:59:49+00:00,1399153605588836354,As Australia begins the long journey to post-C...,LexiconDigital_,1,australia begins long journey postcovid recove...,en,analytical
1,2021-05-30 23:59:29+00:00,1399153522545942531,"@amyklobuchar @All100Senators Its not over, ev...",bargdaffy161,0,heard long covid hot mess senator,en,noTone
2,2021-05-30 23:59:00+00:00,1399153402643288064,@JoeBarri Because not all people have had the ...,Miss_Naste,0,people opportunity vaccine area high percentag...,en,noTone
3,2021-05-30 23:58:51+00:00,1399153364311724033,COVID shone a spotlight on for-profit long ter...,JSSBennett,0,covid shone spotlight forprofit long term care...,en,sadness
4,2021-05-30 23:58:37+00:00,1399153305218170882,@kiroriwalanuj @swamiyogeshji @c_aashish @bhar...,Vl17262873,2,yes agree anil sensible looks longterm neglect...,en,noTone
...,...,...,...,...,...,...,...,...
97097,2021-09-24 18:37:18+00:00,1441471824500432897,Spent the day in BITS Pilani. Just reached Del...,shantanunandan2,85,spent day bits pilani reached delhi semesterlo...,en,joy
97098,2021-09-24 18:36:30+00:00,1441471622796365824,Coronavirus latest: US’s top medical research ...,COVID19LiveNow,0,coronavirus latest uss medical research agency...,en,noTone
97099,2021-09-24 18:36:25+00:00,1441471599560077316,@csmalls11 @WFKARS Sorry! I am taking back eve...,CowChickEnCow,2,sorry taking tweet massive karenoutburst schoo...,en,sadness
97100,2021-09-24 18:36:21+00:00,1441471585567666176,"2/noted the statement, which doesn’t commit to...",ljfawkes,0,noted statement doesnt commit providing noninf...,en,analytical


In [43]:
df['emotion'].value_counts()

analytical    36530
sadness       19823
noTone        18033
tentative      7959
joy            5958
confident      4791
fear           2610
anger          1398
Name: emotion, dtype: int64

In [76]:
#Output the combine dataset
#df.to_csv('D:\\Course Learning - UM\\Sem 5\\Data Science Project\\Emotion Analysis for Long Covid\\testing clean text\\EmotionLongCovidV3.csv',
#          columns = ['Datetime','Tweet Id', 'Text', 'Username', 'Number of Likes','clean_tweet','emotion'], index= False)

# ——————————————Pre-clean Cycle2————————————

In [3]:
df = pd.read_csv("D:\\Course Learning - UM\\Sem 5\\Data Science Project\\Emotion Analysis for Long Covid\\testing clean text\\EmotionLongCovid.csv")

In [4]:
df

,Datetime,Tweet Id,Text,Username,Number of Likes,clean_tweet,emotion
0,2021-05-30 23:59:49+00:00,1399153605588836354,As Australia begins the long journey to post-C...,LexiconDigital_,1,australia begins long journey postcovid recove...,analytical
1,2021-05-30 23:59:29+00:00,1399153522545942531,"@amyklobuchar @All100Senators Its not over, ev...",bargdaffy161,0,heard long covid hot mess senator,neutral
2,2021-05-30 23:59:00+00:00,1399153402643288064,@JoeBarri Because not all people have had the ...,Miss_Naste,0,people opportunity vaccine area high percentag...,neutral
3,2021-05-30 23:58:51+00:00,1399153364311724033,COVID shone a spotlight on for-profit long ter...,JSSBennett,0,covid shone spotlight forprofit long term care...,sadness
4,2021-05-30 23:58:37+00:00,1399153305218170882,@kiroriwalanuj @swamiyogeshji @c_aashish @bhar...,Vl17262873,2,yes agree anil sensible looks longterm neglect...,neutral
...,...,...,...,...,...,...,...
97093,2021-09-24 18:37:18+00:00,1441471824500432897,Spent the day in BITS Pilani. Just reached Del...,shantanunandan2,85,spent day bits pilani reached delhi semesterlo...,joy
97094,2021-09-24 18:36:30+00:00,1441471622796365824,Coronavirus latest: US’s top medical research ...,COVID19LiveNow,0,coronavirus latest uss medical research agency...,neutral
97095,2021-09-24 18:36:25+00:00,1441471599560077316,@csmalls11 @WFKARS Sorry! I am taking back eve...,CowChickEnCow,2,sorry taking tweet massive karenoutburst schoo...,sadness
97096,2021-09-24 18:36:21+00:00,1441471585567666176,"2/noted the statement, which doesn’t commit to...",ljfawkes,0,noted statement doesnt commit providing noninf...,analytical


In [5]:
# Pre-clean For round2
from nltk.corpus import stopwords
NLTK_stop_words_list = stopwords.words('english')
custom_stop_word_list = ['you know','i mean','yo','dude','couldnt','cant','dont','doesnt','youve','im','ive','wasnt',
                         'mightnt','hadnt','hvnt','youre','wouldnt','shouldnt','arent','isnt','werent','youll','its',
                         'thats', 'covid', 'long', 'vaccine', 'know', 'people', 'amp', 'time', 'need', 'like', 
                         'year', 'term', 'risk', 'vaccinate', 'symptom', 'work', 'gonna', 'gon na'
                        ]
final_stopword_list = custom_stop_word_list + NLTK_stop_words_list
df.clean_tweet = df.clean_tweet.apply(lambda x: ' '.join(term for term in x.split() if term not in final_stopword_list))

In [6]:
import nltk
from nltk.corpus import wordnet

#https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
# Part of speech Tagging
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [7]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Lemmatization instead of stemming

lemmatizer = WordNetLemmatizer()

def clean_text_round2(text):
    tokens = nltk.word_tokenize(text)
    tagged = nltk.pos_tag(tokens)
    #print(tagged)
    full_text = ''
    for word, tag in tagged:
        wntag = get_wordnet_pos(tag)
        if wntag is None:
            lemma = lemmatizer.lemmatize(word)
        else:
            lemma = lemmatizer.lemmatize(word, pos=wntag)
        full_text += lemma + ' '
    return full_text
round2 = lambda x: clean_text_round2(x)

In [8]:
df.clean_tweet = df.clean_tweet.apply(round2)

In [9]:
# Export dataset to pkl format
import pickle
df.to_pickle("emotion_datasetV2.pkl")